# Try DreamBooth with my profile photos
Note: This notebook isn't runnable since there are no training images in the repo.
Please use the repo just for reference.

## Overview
Fine-tuning the Stable Diffusion 1.5 model with my 30 profile photos.
Training the model with the setting of 14 epochs and 28 epochs.

## Example repo
https://github.com/huggingface/diffusers/tree/main/examples/dreambooth

## Prerequisites
- VRAM 16GB GPU

## Setup environment
```bash
git clone git@github.com:huggingface/diffusers.git
pip install -U -r diffusers/examples/dreambooth/requirements.txt
pip install git+https://github.com/huggingface/diffusers

pip install bitsandbytes # to use --use_8bit_adam option
pip install scipy

accelerate config default
```

## Prep for HuggingFace repo
Get hugging face token with write permission from
https://huggingface.co/settings/tokens

In [ ]:
!conda activate sd
!huggingface-cli login #you need to get your token with write permission
!cd diffusers/examples/dreambooth

# Train (14 epoch)
Use `--gradient_checkpointing` and `--use_8bit_adam` option to train on 16GB VRAM GPU.

```bash
export MODEL_NAME="runwayml/stable-diffusion-v1-5"
export INSTANCE_DIR="photo-shohei"
export OUTPUT_DIR="dreambooth-shohei-14"

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="Full body profile photo of a man shohei in various situation" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --push_to_hub \
  --gradient_checkpointing \
  --use_8bit_adam
```

# Train (28 epochs)
`max_train_steps` overrides `num_train_epochs`

```bash
export MODEL_NAME="runwayml/stable-diffusion-v1-5"
export INSTANCE_DIR="photo-shohei"
export OUTPUT_DIR="dreambooth-shohei-28epochs"

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="Full body profile photo of the same person in various situation" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --push_to_hub \
  --gradient_checkpointing \
  --use_8bit_adam \
  --num_train_epochs 28
  ```

# Inference
Change the `model_id` and `prompt` strings to change the model and prompt.

In [37]:
import os
from diffusers import StableDiffusionPipeline
import torch

model_id = "shohei1029/dreambooth-shohei-28"
prompt = "Profile image of shohei in Ukiyoe style"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

output_name = prompt.replace(" ", "_")
output_dir_ = model_id.split("/")[-1]
output_dir = f"outputs/{output_dir_}/"
os.makedirs(output_dir, exist_ok=True)

image.save(f"{output_dir}/{output_name}.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]